In [123]:
import random
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD

In [103]:
lemmatizer = WordNetLemmatizer()

print(lemmatizer)

<WordNetLemmatizer>


In [104]:
words=[]        
documents = []
classes = []
ignore_words = ['?', '!']

In [105]:
data_file = open('data.json').read()
intents = json.loads(data_file)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Bonjour', 'Comment ça va', "Y a-t-il quelqu'un ?", 'Salut', 'Hola', 'Bonjour', 'Bonne journée'], 'responses': ['Salut, comment puis-je vous aider ?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Au revoir', 'À plus tard', 'Au revoir', "C'était agréable de discuter avec vous, au revoir", 'À la prochaine'], 'responses': ['Au revoir ! Revenez bientôt.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Merci', 'Merci beaucoup', "C'est utile", 'Génial, merci', "Merci de m'avoir aidé"], 'responses': ['Ravi de pouvoir aider !'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Veuillez me donner plus d'informations"], 'context': ['']}, {'tag': 'options', 'patterns': ["Comment pourriez-vous m'aider avec la faculté des sciences Ibn Tofail à Kénitra ?", "Qu'est-ce que vous pouvez faire pour m'aider avec la faculté des sciences Ibn Tofail à Kénitra ?", 'Quelle aide pouvez-vous fournir concernant la faculté des sciences Ib

In [106]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        #add documents in the corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [107]:
print(documents)

[(['Bonjour'], 'greeting'), (['Comment', 'ça', 'va'], 'greeting'), (['Y', 'a-t-il', "quelqu'un", '?'], 'greeting'), (['Salut'], 'greeting'), (['Hola'], 'greeting'), (['Bonjour'], 'greeting'), (['Bonne', 'journée'], 'greeting'), (['Au', 'revoir'], 'goodbye'), (['À', 'plus', 'tard'], 'goodbye'), (['Au', 'revoir'], 'goodbye'), (["C'était", 'agréable', 'de', 'discuter', 'avec', 'vous', ',', 'au', 'revoir'], 'goodbye'), (['À', 'la', 'prochaine'], 'goodbye'), (['Merci'], 'thanks'), (['Merci', 'beaucoup'], 'thanks'), (["C'est", 'utile'], 'thanks'), (['Génial', ',', 'merci'], 'thanks'), (['Merci', 'de', "m'avoir", 'aidé'], 'thanks'), (['Comment', 'pourriez-vous', "m'aider", 'avec', 'la', 'faculté', 'des', 'sciences', 'Ibn', 'Tofail', 'à', 'Kénitra', '?'], 'options'), (["Qu'est-ce", 'que', 'vous', 'pouvez', 'faire', 'pour', "m'aider", 'avec', 'la', 'faculté', 'des', 'sciences', 'Ibn', 'Tofail', 'à', 'Kénitra', '?'], 'options'), (['Quelle', 'aide', 'pouvez-vous', 'fournir', 'concernant', 'la', '

In [108]:
# return the base or dictionary form of a word, which is known as the lemma, lower case
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words

['bonjour',
 'comment',
 'ça',
 'va',
 'y',
 'a-t-il',
 "quelqu'un",
 'salut',
 'hola',
 'bonjour',
 'bonne',
 'journée',
 'au',
 'revoir',
 'à',
 'plus',
 'tard',
 'au',
 'revoir',
 "c'était",
 'agréable',
 'de',
 'discuter',
 'avec',
 'vous',
 ',',
 'au',
 'revoir',
 'à',
 'la',
 'prochaine',
 'merci',
 'merci',
 'beaucoup',
 "c'est",
 'utile',
 'génial',
 ',',
 'merci',
 'merci',
 'de',
 "m'avoir",
 'aidé',
 'comment',
 'pourriez-vous',
 "m'aider",
 'avec',
 'la',
 'faculté',
 'de',
 'science',
 'ibn',
 'tofail',
 'à',
 'kénitra',
 "qu'est-ce",
 'que',
 'vous',
 'pouvez',
 'faire',
 'pour',
 "m'aider",
 'avec',
 'la',
 'faculté',
 'de',
 'science',
 'ibn',
 'tofail',
 'à',
 'kénitra',
 'quelle',
 'aide',
 'pouvez-vous',
 'fournir',
 'concernant',
 'la',
 'faculté',
 'de',
 'science',
 'ibn',
 'tofail',
 'à',
 'kénitra',
 'comment',
 'pouvez-vous',
 'être',
 'utile',
 'pour',
 'répondre',
 'à',
 'me',
 'question',
 'sur',
 'la',
 'faculté',
 'de',
 'science',
 'ibn',
 'tofail',
 'à',

In [109]:
# remove duplicates set()
words = sorted(list(set(words)))

In [110]:
# sort classes
classes = sorted(list(set(classes)))

In [111]:
pickle.dump(words, open('texts.pkl','wb'))

In [112]:
pickle.dump(classes, open('labels.pkl','wb'))

In [113]:
# create our training data
training = []

# for our output
output_empty = [0] * len(classes)
output_empty

[0, 0, 0, 0, 0, 0]

In [114]:
b = True
# training set, bag of words for each sentence
for doc in documents:

    # initialize our bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

    if b : 
        print(len(words))
        print(len(bag))
        b = False
    print(bag)
    print(output_row)

72
72
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [115]:
training

[[[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 1, 0, 0, 0]],
 [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0],
  [0, 0, 1, 0, 0, 0]],
 [[0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [116]:
# Shuffling: It shuffles (randomly reorders) the elements of the list training in place.
random.shuffle(training)

In [166]:
print(len(training))
print(len(training[0]))
print(len(training[0][0]))
print(len(training[0][1]))

30
2
72
6


In [118]:
# training = np.array(training)

In [176]:
# create train and test lists. X - patterns, Y - intents
train_x = [training[x][0] for x in range(len(training))]
train_y = [training[x][1] for x in range(len(training))]

In [181]:
print(len(train_x))
print(len(train_x[0]))

30
72


In [182]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

/home/sonix/docc/programming/pypr/virenv/uitbot/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [183]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/home/sonix/docc/programming/pypr/virenv/uitbot/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [184]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.1338 - loss: 1.8701   
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1205 - loss: 1.7174     
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2767 - loss: 1.6770 
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5114 - loss: 1.5311
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4195 - loss: 1.4126 
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5943 - loss: 1.4888  
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6462 - loss: 1.3409 
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7910 - loss: 1.0898  
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6281 - loss: 1.3066 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7905 - loss: 1.0897 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7462 - loss: 1.0008 
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0

model created


In [1]:
import nltk
from nltk.stem import WordNetLemmatizer

import json
import random
import pickle
import numpy as np

from tensorflow.keras.models import load_model

from flask import Flask, render_template, request

2024-06-22 11:23:42.083473: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-22 11:23:42.094959: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-22 11:23:42.228912: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 11:23:45.544011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
nltk.download('popular')
lemmatizer = WordNetLemmatizer()

model = load_model('my_model.keras')

intents = json.loads(open('data.json').read())

words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/sonix/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/sonix/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/sonix/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/sonix/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/sonix/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/sonix/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

In [3]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)

    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words
    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [4]:
# bag of word
def bow(sentence, words, show_details = True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [5]:
def predict_class(msg, model):
    # filter out predictions below a threshold
    p = bow(msg, words, show_details = False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [35]:
msg1 = "Bonjour!"
msg2 = "comment je peut contacter universiter ?"
msg3 = "lister moi les actualites de l'universite ?"

p = bow(msg3, words, show_details = False)
print(np.array(p))

res = model.predict(np.array([p]))
print()
print(res)

res = res[0]
print()
print(res)

ERROR_THRESHOLD = 0.25
results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

print()
print(results)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step

[[0.04799196 0.00494577 0.51345146 0.00251917 0.07904186 0.35204983]]

[0.04799196 0.00494577 0.51345146 0.00251917 0.07904186 0.35204983]

[[2, 0.51345146], [5, 0.35204983]]


In [36]:
results.sort(key=lambda x: x[1], reverse=True)
print(results)

[[2, 0.51345146], [5, 0.35204983]]


In [37]:
# sort by strength of probability
results.sort(key=lambda x: x[1], reverse=True)
return_list = []
for r in results:
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
print(return_list)

[{'intent': 'greeting', 'probability': '0.51345146'}, {'intent': 'uni_contact', 'probability': '0.35204983'}]


In [38]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [43]:
return_list

[{'intent': 'greeting', 'probability': '0.51345146'},
 {'intent': 'uni_contact', 'probability': '0.35204983'}]

In [46]:
tag = return_list[0]['intent']
list_of_intents = intents['intents']
# print(list_of_intents)
for i in list_of_intents:
    if(i['tag']== tag):
        result = random.choice(i['responses'])
        break

print(result)

Salut, comment puis-je vous aider ?
